# Running CEDS Scenarios

In this notebook we document how to process and run data from the CEDS database.

In [1]:
%matplotlib inline

from os import listdir
from os.path import join, dirname
from pprint import pprint

import pandas as pd
import pyam
from pyam.utils import LONG_IDX
import pint
from pint.pandas_interface import PintArray
from pint.errors import DimensionalityError

import pymagicc
from pymagicc.io import MAGICCData

import matplotlib.pyplot as plt
plt.style.use('bmh') 

import expectexception

<IPython.core.display.Javascript object>

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
TEST_DATA_PATH = join("..", "tests", "test_data")

## Reading in a CEDS csv

To read in CEDS csv's, we make use of the `pyam` library which is specifically designed for this purpose.

In [4]:
def read_ceds_csv(file_to_read):
    return pyam.IamDataFrame(
        data=file_to_read,
        encoding="utf-8"
    )

ceds_pyam_df = read_ceds_csv(join(TEST_DATA_PATH, "ceds-format-example.csv"))
ceds_pyam_df  # this just shows the type of ceds_pyam_df
ceds_pyam_df.data  # this returns the underlying DataFrame

INFO:root:Reading `../tests/test_data/ceds-format-example.csv`


,model,scenario,region,variable,unit,year,value
0,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5ASIA,Emissions|BC,Mt BC/yr,2015,36.398915
91,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5LAM,Emissions|BC,Mt BC/yr,2015,26.494276
174,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5MAF,Emissions|BC,Mt BC/yr,2015,30.456913
257,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5OECD,Emissions|BC,Mt BC/yr,2015,32.513038
348,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5REF,Emissions|BC,Mt BC/yr,2015,18.561100
439,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,World,Emissions|BC,Mt BC/yr,2015,152.232300
2276,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5ASIA,Emissions|BC,Mt BC/yr,2020,26.645995
2367,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5LAM,Emissions|BC,Mt BC/yr,2020,4.195137
2450,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5MAF,Emissions|BC,Mt BC/yr,2020,24.990606
2533,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,R5OECD,Emissions|BC,Mt BC/yr,2020,20.422119


## Checking an `IamDataFrame`

It is very easy to check that the sum of a given variable's sub-categories is equal to its declared total and that the sum of regions gives the world total.

We show how in the next cell.

In [5]:
# show check_internal_consistency method here

## Reshaping an `IamDataFrame`

Here we show how to reshape an `IamDataFrame` to get it into the format expected by `openscm` so we can then write files with the data in it.

Note: we normally want to take this step last, after we have done all our aggregation etc., as it means that we no longer have an `IamDataFrame` and can't use all the helpful tools it provides any more.

In [6]:
def reshape_pyam_df_to_openscm_df(pyam_df):
    raw_df = pyam_df.data
    
    reindexed_df = raw_df.set_index(
        ["model", "scenario", "region", "variable", "unit", "year"]
    ).unstack().T
    
    years = reindexed_df.index.get_level_values("year")
    if (years % 1 == 0).all() :
        reindexed_df.index = years.astype(int)
    reindexed_df.index.name = "YEAR"
    
    models = reindexed_df.columns.get_level_values("model")
    scenarios = reindexed_df.columns.get_level_values("scenario")
    regions = reindexed_df.columns.get_level_values("region")
    variables = reindexed_df.columns.get_level_values("variable")
    units = reindexed_df.columns.get_level_values("unit")
    todos = ["SET"] * len(units)
    
    reindexed_df.columns = pd.MultiIndex.from_arrays(
        [models, scenarios, variables, todos, units, regions],
        names=("MODEL", "SCENARIO", "VARIABLE", "TODO", "UNITS", "REGION"),
    )
    
    return reindexed_df

In [7]:
reshape_pyam_df_to_openscm_df(ceds_pyam_df)

MODEL    MODEL-NAME-HYPHENS                                          \
SCENARIO SCENARIO-A-B-CDE-2                                           
VARIABLE       Emissions|BC Emissions|BC|Agricultural Waste Burning   
TODO                    SET                                     SET   
UNITS              Mt BC/yr                                Mt BC/yr   
REGION               R5ASIA                                  R5ASIA   
YEAR                                                                  
2015              36.398915                                3.974090   
2020              26.645995                                2.737676   
2030              36.275098                                9.717128   
2040              32.112875                                8.026567   
2050              32.827797                                1.968237   
2060              47.318307                                3.889015   
2070              30.100152                                2.213773   
2080              27.344638                                7.074890   
2090              36.352506                                7.606149   
2100              31.187290                                4.797244   

MODEL                                                            \
SCENARIO                                                          
VARIABLE Emissions|BC|Energy Sector Emissions|BC|Forest Burning   
TODO                            SET                         SET   
UNITS                      Mt BC/yr                    Mt BC/yr   
REGION                       R5ASIA                      R5ASIA   
YEAR                                                              
2015                       1.310619                    2.652261   
2020                       8.896164                    3.540554   
2030                       3.933159                   -1.479242   
2040                       6.339154                    1.199015   
2050                       8.799231                    0.975675   
2060                       1.607069                    4.169055   
2070                       4.971571                    4.518712   
2080                      -1.760939                    8.024338   
2090                      -1.632714                    2.740800   
2100                       2.411164                    1.578012   

MODEL                                                                   \
SCENARIO                                                                 
VARIABLE Emissions|BC|Grassland Burning Emissions|BC|Industrial Sector   
TODO                                SET                            SET   
UNITS                          Mt BC/yr                       Mt BC/yr   
REGION                           R5ASIA                         R5ASIA   
YEAR                                                                     
2015                           6.632639                       6.185669   
2020                          -1.279472                      -1.203520   
2030                           5.246591                       1.119544   
2040                           4.793242                      -0.491045   
2050                           4.584317                       7.386991   
2060                           7.077807                       6.433131   
2070                           7.762036                       1.085432   
2080                           0.722233                       4.163659   
2090                           1.581080                       5.675770   
2100                           4.848120                       1.165334   

MODEL                                                                         \
SCENARIO                                                                       
VARIABLE Emissions|BC|Peat Burning Emissions|BC|Residential Commercial Other   
TODO                           SET                                       SET   
UNITS                     Mt BC/yr                                  Mt BC

## Super brief intro to pyam

The `pyam` library provides some very natural ways of filtering their DataFrames. These are detailed in [their tutorial](https://github.com/IAMconsortium/pyam/blob/master/tutorial/pyam_first_steps.ipynb). Here we use them to help convert IAM data into the emissions variables, regions and units used by openscm and MAGICC.

In [8]:
tdf = ceds_pyam_df.filter(
    level=1,
    model="MODEL-NAME-HYPHENS",
    scenario="SCENARIO-A-B-CDE-2",
    region="World",
)
tdf.variables()
tdf.regions()
tdf.data.head()

0            Emissions|BC
1          Emissions|C2F6
2          Emissions|CCl4
3           Emissions|CF4
4        Emissions|CFC-11
5       Emissions|CFC-113
6       Emissions|CFC-114
7       Emissions|CFC-115
8        Emissions|CFC-12
9           Emissions|CH3
10      Emissions|CH3CCl3
11        Emissions|CH3Cl
12          Emissions|CH4
13           Emissions|CO
14          Emissions|CO2
15    Emissions|HCFC-141b
16    Emissions|HCFC-142b
17      Emissions|HCFC-22
18          Emissions|HFC
19    Emissions|Halon1202
20    Emissions|Halon1211
21    Emissions|Halon1301
22    Emissions|Halon2402
23          Emissions|N2O
24          Emissions|NH3
25          Emissions|NOx
26           Emissions|OC
27          Emissions|SF6
28       Emissions|Sulfur
29          Emissions|VOC
Name: variable, dtype: object

0    World
Name: region, dtype: object

,model,scenario,region,variable,unit,year,value
439,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,World,Emissions|BC,Mt BC/yr,2015,152.232300
2715,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,World,Emissions|BC,Mt BC/yr,2020,127.122154
4991,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,World,Emissions|BC,Mt BC/yr,2030,177.813834
7267,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,World,Emissions|BC,Mt BC/yr,2040,218.627907
9543,MODEL-NAME-HYPHENS,SCENARIO-A-B-CDE-2,World,Emissions|BC,Mt BC/yr,2050,219.680082


## Converting CEDS data to openscm data

In [9]:
# ceds openscm mapping
ceds_openscm_var_mapping = {
    "Aircraft": ["Aircraft"],
    "International Shipping": ["International Shipping"],
    "AFOLULUC": ["Agricultural Waste Burning", "Agriculture", 
                 "Forest Burning", "Grassland Burning", 
                 "Peat Burning", "Aggregate - Agriculture and LUC"],
    "Fossil": ["Energy Sector", "Industrial Sector", 
               "Residential Commercial Other", 
               "Solvents Production and Application", 
               "Transportation Sector", "Waste"]
}
# need better name for this
TMP_INDEX = ['model', 'scenario', 'region', 'year', 'unit']

In [10]:
def convert_ceds_to_openscm_variable(ceds_var):
    raw_ceds_var = ceds_var.replace("Emissions|", "")
    
    special_cases = {
        "Sulfur": "SOX",
    }
    
    if raw_ceds_var in special_cases:
        raw_var = special_cases[raw_ceds_var]
    else:
        raw_var = raw_ceds_var.replace("-", "").upper()
        
    return "Emissions|" + raw_var

def convert_ceds_df_to_openscm_df(pyam_df):
    openscm_df = pyam.IamDataFrame(data=pyam_df.data.copy())
    openscm_df.data.region = openscm_df.data.region.str.upper()
    
    output_df = openscm_df.filter(level=1,).data
    output_df.variable = output_df.variable.apply(convert_ceds_to_openscm_variable)
    
    openscm_df = openscm_df.filter(
        level='1-', 
        keep=False
    )
    
    handled_vars = []
    metadata = {}
    for variable in openscm_df.variables():
        base_var = "|".join(variable.split("|")[:2])
        
        for category, suffixes in ceds_openscm_var_mapping.items():
            openscm_var = "{}|{}".format(
                convert_ceds_to_openscm_variable(variable.split("|")[1]),
                category,
            )
            if openscm_var in handled_vars:
                continue
            handled_vars.append(openscm_var)
            
            contrib_vars = ["{}|{}".format(base_var, s) for s in suffixes]
            
            var_cat_df = openscm_df.data[openscm_df.data.variable.isin(contrib_vars)]
            var_cat_df = pd.DataFrame(var_cat_df.groupby(TMP_INDEX).sum()['value'])
            var_cat_df = pd.concat([var_cat_df], keys=[openscm_var], names=['variable'])

            output_df = pd.concat([output_df, var_cat_df.reset_index()], sort=False)

            metadata[openscm_var] = "Sum of {}".format(", ".join(contrib_vars))
    
    return pyam.IamDataFrame(data=output_df), metadata

In [11]:
openscm_df, metadata = convert_ceds_df_to_openscm_df(ceds_pyam_df)

pprint([v for v in openscm_df.variables()])
pprint([r for r in openscm_df.regions()])
reshape_pyam_df_to_openscm_df(openscm_df)

['Emissions|BC',
 'Emissions|BC|AFOLULUC',
 'Emissions|BC|Aircraft',
 'Emissions|BC|Fossil',
 'Emissions|BC|International Shipping',
 'Emissions|C2F6',
 'Emissions|CCL4',
 'Emissions|CF4',
 'Emissions|CFC11',
 'Emissions|CFC113',
 'Emissions|CFC114',
 'Emissions|CFC115',
 'Emissions|CFC12',
 'Emissions|CH3',
 'Emissions|CH3CCL3',
 'Emissions|CH3CL',
 'Emissions|CH4',
 'Emissions|CH4|AFOLULUC',
 'Emissions|CH4|Fossil',
 'Emissions|CH4|International Shipping',
 'Emissions|CO',
 'Emissions|CO2',
 'Emissions|CO2|AFOLULUC',
 'Emissions|CO2|Aircraft',
 'Emissions|CO2|Fossil',
 'Emissions|CO2|International Shipping',
 'Emissions|CO|AFOLULUC',
 'Emissions|CO|Aircraft',
 'Emissions|CO|Fossil',
 'Emissions|CO|International Shipping',
 'Emissions|HALON1202',
 'Emissions|HALON1211',
 'Emissions|HALON1301',
 'Emissions|HALON2402',
 'Emissions|HCFC141B',
 'Emissions|HCFC142B',
 'Emissions|HCFC22',
 'Emissions|HFC',
 'Emissions|N2O',
 'Emissions|NH3',
 'Emissions|NH3|AFOLULUC',
 'Emissions|NH3|Aircra

MODEL    MODEL-NAME-HYPHENS                                            \
SCENARIO SCENARIO-A-B-CDE-2                                             
VARIABLE       Emissions|BC Emissions|BC|AFOLULUC Emissions|BC|Fossil   
TODO                    SET                   SET                 SET   
UNITS              Mt BC/yr              Mt BC/yr            Mt BC/yr   
REGION               R5ASIA                R5ASIA              R5ASIA   
YEAR                                                                    
2015              36.398915             19.897554           16.501361   
2020              26.645995              9.260125           17.385870   
2030              36.275098             22.694679           13.580419   
2040              32.112875             13.432291           18.680585   
2050              32.827797             13.065412           19.762386   
2060              47.318307             23.685064           23.633243   
2070              30.100152             13.515651           16.584501   
2080              27.344638             16.320317           11.024320   
2090              36.352506             19.514986           16.837519   
2100              31.187290             13.959489           17.227801   

MODEL                                                               \
SCENARIO                                                             
VARIABLE Emissions|CH4 Emissions|CH4|AFOLULUC Emissions|CH4|Fossil   
TODO               SET                    SET                  SET   
UNITS        Mt CH4/yr              Mt CH4/yr            Mt CH4/yr   
REGION          R5ASIA                 R5ASIA               R5ASIA   
YEAR                                                                 
2015         33.186689              11.676655            21.510034   
2020         49.676666              34.772684            14.903982   
2030         46.979559              31.844023            15.135537   
2040         44.691875              11.953285            32.738590   
2050         30.754564              18.804239            11.950325   
2060         45.037278              10.391316            34.645962   
2070         40.648090              13.925268            26.722822   
2080         47.319430              31.660236            15.659194   
2090         35.504855              13.973206            21.531649   
2100         38.099801              11.695808            26.403994   

MODEL                                                     \
SCENARIO                                                   
VARIABLE Emissions|CO Emissions|CO2 Emissions|CO2|Fossil   
TODO              SET           SET                  SET   
UNITS        Mt CO/yr     Mt CO2/yr            Mt CO2/yr   
REGION         R5ASIA        R5ASIA               R5ASIA   
YEAR                                                       
2015        27.647071     36.300442            36.300442   
2020        37.980337     18.820329            18.820329   
2030        56.035339     19.323918            19.323918   
2040        28.729780     29.048607            29.048607   
2050        27.664517     22.267156            22.267156   
2060        29.938876     21.045728            21.045728   
2070        26.393159     32.946365            32.946365   
2080        44.438492     22.693831            22.693831   
2090        55.683231     16.846655            16.846655   
2100        38.403976     15.680066            15.680066   

MODEL                                          ...                   \
SCENARIO                                       ...                    
VARIABLE Emissions|CO|AFOLULUC                 ...                    
TODO                       SET                 ...                    
UNITS                 Mt CO/yr                 ...                    
REGION                  R5ASIA                 ...                    
YEAR                                           ...                    
2015                  3.371103                 ..

## Converting openscm data to MAGICC data

Here we show how to then convert an openscm data table to MAGICC data.

In [12]:
def convert_openscm_to_magicc_variable(ceds_variable):
    category_codes = {
        "Aircraft": "AIR",
        "International Shipping": "SHIP",
        "AFOLULUC": "B",
        "Fossil": "I",
    }
    
    special_cases = {
        "VOC": "NMVOC",
    }
    
    # Improvement: do this with regexp
    species = ceds_variable.split("|")[1]
    if species in special_cases:
        species = special_cases[species]
        
    try:
        category = ceds_variable.split("|")[2]
        category_code = category_codes[category]
    except IndexError:
        category_code = ""
    
    return "{}{}_EMIS".format(species, category_code)
    
def convert_openscm_to_magicc_df(openscm_df):
    magicc_df = pyam.IamDataFrame(data=openscm_df.data.copy())
    magicc_df.data.variable = magicc_df.data.variable.apply(convert_openscm_to_magicc_variable)
    # unit conversions
    return magicc_df

In [13]:
magicc_df = convert_openscm_to_magicc_df(openscm_df)
reshape_pyam_df_to_openscm_df(magicc_df)

MODEL    MODEL-NAME-HYPHENS                                              \
SCENARIO SCENARIO-A-B-CDE-2                                               
VARIABLE           BCB_EMIS   BCI_EMIS    BC_EMIS  CH4B_EMIS  CH4I_EMIS   
TODO                    SET        SET        SET        SET        SET   
UNITS              Mt BC/yr   Mt BC/yr   Mt BC/yr  Mt CH4/yr  Mt CH4/yr   
REGION               R5ASIA     R5ASIA     R5ASIA     R5ASIA     R5ASIA   
YEAR                                                                      
2015              19.897554  16.501361  36.398915  11.676655  21.510034   
2020               9.260125  17.385870  26.645995  34.772684  14.903982   
2030              22.694679  13.580419  36.275098  31.844023  15.135537   
2040              13.432291  18.680585  32.112875  11.953285  32.738590   
2050              13.065412  19.762386  32.827797  18.804239  11.950325   
2060              23.685064  23.633243  47.318307  10.391316  34.645962   
2070              13.515651  16.584501  30.100152  13.925268  26.722822   
2080              16.320317  11.024320  27.344638  31.660236  15.659194   
2090              19.514986  16.837519  36.352506  13.973206  21.531649   
2100              13.959489  17.227801  31.187290  11.695808  26.403994   

MODEL                                                               ...      \
SCENARIO                                                            ...       
VARIABLE   CH4_EMIS  CO2I_EMIS   CO2_EMIS   COB_EMIS   COI_EMIS     ...       
TODO            SET        SET        SET        SET        SET     ...       
UNITS     Mt CH4/yr  Mt CO2/yr  Mt CO2/yr   Mt CO/yr   Mt CO/yr     ...       
REGION       R5ASIA     R5ASIA     R5ASIA     R5ASIA     R5ASIA     ...       
YEAR                                                                ...       
2015      33.186689  36.300442  36.300442   3.371103  24.275968     ...       
2020      49.676666  18.820329  18.820329  23.556976  14.423361     ...       
2030      46.979559  19.323918  19.323918  20.931029  35.104309     ...       
2040      44.691875  29.048607  29.048607   7.942382  20.787398     ...       
2050      30.754564  22.267156  22.267156  14.235618  13.428900     ...       
2060      45.037278  21.045728  21.045728  15.888153  14.050723     ...       
2070      40.648090  32.946365  32.946365   6.795837  19.597322     ...       
2080      47.319430  22.693831  22.693831  21.066322  23.372171     ...       
2090      35.504855  16.846655  16.846655  24.298416  31.384815     ...       
2100      38.099801  15.680066  15.680066  16.972965  21.431011     ...       

MODEL             NAME-MODEL-2                                                \
SCENARIO SCENARIO-NAME-HYPHENS                                                 
VARIABLE              OCB_EMIS    OCI_EMIS OCSHIP_EMIS     OC_EMIS  SF6_EMIS   
TODO                       SET         SET         SET         SET       SET   
UNITS                 Mt OC/yr    Mt OC/yr    Mt OC/yr    Mt OC/yr kt SF6/yr   
REGION                   WORLD       WORLD       WORLD       WORLD     WORLD   
YEAR                                                                           
2015                 57.318664   74.312308    3.802944  142.979269  1.800539   
2020                 73.126747   99.627257    2.748180  180.471025 -1.997450   
2030                 33.132244   76.885968    5.380791  123.248619  8.141167   
2040                 94.125676   93.139837    0.911188  186.454927  3.737162   
2050                 97.124686  104.350798    2.171461  211.132081  1.850793   
2060                 58.928189   81.952714   -0.394555  145.625701  1.536665   
2070                 60.117122  103.015167    4.660071  174.346000  6.502613   
2080                 68.602552  111.527680    7.894283  191.766940 -1.059791   
2090                 64.963427   88.351171   -1.801544  154.051650  0.180659   
2100                 79.258432  127.666884    1.040398  213.115027  2.432472   

MODEL              

### Converting to SCEN7 format

In [14]:
def convert_bunkers_to_magicc6_variable(bunker_variable):
    return bunker_variable.replace("SHIP", "I").replace("AIR", "I")

def get_bunkers_df_from_magicc_df(magicc_df):
    ship_df = magicc_df.filter(variable="*SHIP*").data
    ship_df.variable = ship_df.variable.apply(convert_bunkers_to_magicc6_variable)
    ship_df.region = "BUNKERS"
    ship_df.set_index(LONG_IDX, inplace=True)
    
    air_df = magicc_df.filter(variable="*AIR*").data
    air_df.variable = air_df.variable.apply(convert_bunkers_to_magicc6_variable)
    air_df.region = "BUNKERS"
    air_df.set_index(LONG_IDX, inplace=True)

    bunkers_df = ship_df + air_df
    bunkers_df.reset_index(inplace=True)
    
    return bunkers_df

def magicc_df_to_scen7_df(magicc_df):
    scen7_df = magicc_df.filter(
        variable=["*SHIP*", "*AIR*"], 
        keep=False
    )
    # strip out all the variables with breakdown
    # data available
    for variable in scen7_df.variables():
        if variable.endswith(("I_EMIS", "B_EMIS")):
            continue

        if variable.replace("_EMIS", "I_EMIS") in scen7_df.variables().tolist():
            scen7_df = scen7_df.filter(
                variable=variable, 
                keep=False
            )
    
    # to dicuss with Malte, should we do this 
    # given I don't think it matters:
    # - add in N2O breakdown
    # - add in CO2B breakdown
    
    scen7_df = scen7_df.data
    bunkers_df = get_bunkers_df_from_magicc_df(magicc_df)
    
    scen7_df = pd.concat([scen7_df, bunkers_df])
    
    return pyam.IamDataFrame(data=scen7_df)

In [15]:
scen7_df = reshape_pyam_df_to_openscm_df(
    magicc_df_to_scen7_df(magicc_df)
)

#### Write SCEN7 files

In [16]:
scen7_df

MODEL    MODEL-NAME-HYPHENS                                               \
SCENARIO SCENARIO-A-B-CDE-2                                                
VARIABLE           BCI_EMIS  CO2I_EMIS   COI_EMIS  NH3I_EMIS NMVOCI_EMIS   
TODO                    SET        SET        SET        SET         SET   
UNITS              Mt BC/yr  Mt CO2/yr   Mt CO/yr  Mt NH3/yr   Mt VOC/yr   
REGION              BUNKERS    BUNKERS    BUNKERS    BUNKERS     BUNKERS   
YEAR                                                                       
2015               7.808057   8.855288   7.899835  -0.449890   12.153599   
2020              15.127783   8.139339   8.753331  11.475173   11.208207   
2030              13.879839   8.051441  10.233600  10.392513   10.835994   
2040               3.586545  12.545369  15.570558  11.223494    4.095413   
2050               2.919318  10.074329   8.363170   7.984235    8.320671   
2060               5.067740   2.421339   2.760328  11.695897   10.646372   
2070               2.749164  17.814434   6.712114   9.780849    4.997370   
2080               1.171113   8.984201   6.792964   4.856507    0.024543   
2090               7.432437   8.771807   0.790272  11.612190    5.530516   
2100               0.486817  -0.499016   4.170258   6.492259   11.521443   

MODEL                                                               ...      \
SCENARIO                                                            ...       
VARIABLE  NOXI_EMIS   OCI_EMIS  SOXI_EMIS   BCB_EMIS   BCI_EMIS     ...       
TODO            SET        SET        SET        SET        SET     ...       
UNITS     Mt NOx/yr   Mt OC/yr  Mt SO2/yr   Mt BC/yr   Mt BC/yr     ...       
REGION      BUNKERS    BUNKERS    BUNKERS     R5ASIA     R5ASIA     ...       
YEAR                                                                ...       
2015       4.195142   6.782917   7.912581  19.897554  16.501361     ...       
2020      11.464866   9.448994   2.530590   9.260125  17.385870     ...       
2030      11.615368   7.955180   9.769511  22.694679  13.580419     ...       
2040      14.403432   1.174812  14.115141  13.432291  18.680585     ...       
2050       4.295391   8.593765   5.580531  13.065412  19.762386     ...       
2060      10.441905  15.279038  17.651168  23.685064  23.633243     ...       
2070       6.375483  -0.443323  18.218800  13.515651  16.584501     ...       
2080       7.796152   0.542668   5.841155  16.320317  11.024320     ...       
2090       4.741502   4.976972  17.317741  19.514986  16.837519     ...       
2100      13.085589   4.711210   6.387360  13.959489  17.227801     ...       

MODEL             NAME-MODEL-2                                      \
SCENARIO SCENARIO-NAME-HYPHENS                                       
VARIABLE             NH3I_EMIS NMVOCB_EMIS NMVOCI_EMIS   NOXB_EMIS   
TODO                       SET         SET         SET         SET   
UNITS                Mt NH3/yr   Mt VOC/yr   Mt VOC/yr   Mt NOx/yr   
REGION                   WORLD       WORLD       WORLD       WORLD   
YEAR                                                                 
2015                 84.934645   68.916762  101.894351   76.693070   
2020                 97.813641   58.357976  141.768481   84.262013   
2030                 83.509742   65.510687  142.168406  107.920254   
2040                134.193599   70.872486  151.084496   84.982498   
2050                105.194942   70.586460  107.939485   72.949639   
2060                101.349886   55.844966  108.680574   96.519916   
2070                 51.805945   90.416292  117.807967  105.582604   
2080                110.301355   68.895227  121.689800  122.934990   
2090                102.735132   63.649517  126.112733  102.768453   
2100                 77.860083   70.553602  133.198587  100.027460   

MODEL                                                                          
SCENARIO                                                                       
VARIABLE   N

In [17]:
for label, df in scen7_df.groupby(level=["MODEL", "SCENARIO"], axis=1):
    fn = "{}_{}.SCEN7".format(*label)
    df.columns = df.columns.droplevel("MODEL").droplevel("SCENARIO")
    df.index = df.index.astype(int)
    writer = MAGICCData()
    writer.df = df
    writer.metadata = {
        "header": "required for some reason\n\n"
    }
    writer.write(fn)

### Converting to SCEN format

In [18]:
def magicc_df_to_scen_df(magicc_df, world_only=True):
    if not world_only:
        raise NotImplementedError("Neccesary checks not yet included")
        
    scen_emis = [
        v + "_EMIS" 
        for v in pymagicc.definitions.scen_emms_code_1]
    scen_df = magicc_df.filter(
        variable=["*SHIP*", "*AIR*"],
        keep=False
    )
    
    scen_df = scen_df.data
    bunkers_df = get_bunkers_df_from_magicc_df(magicc_df)
    
    scen_df = pd.concat([scen_df, bunkers_df])
    
    scen_df = pyam.IamDataFrame(data=scen_df)
    if world_only:
        scen_df = scen_df.filter(
            region="WORLD"
        )
    
    return scen_df.filter(variable=scen_emis)

In [19]:
scen_iam_df = magicc_df_to_scen_df(magicc_df)
scen_df = reshape_pyam_df_to_openscm_df(scen_iam_df)
scen_iam_df.variables()

0        BC_EMIS
1      C2F6_EMIS
2       CF4_EMIS
3       CH4_EMIS
4      CO2B_EMIS
5      CO2I_EMIS
6        CO_EMIS
7       N2O_EMIS
8       NH3_EMIS
9     NMVOC_EMIS
10      NOX_EMIS
11       OC_EMIS
12      SF6_EMIS
13      SOX_EMIS
Name: variable, dtype: object

In [20]:
for label, df in scen_df.groupby(level=["MODEL", "SCENARIO"], axis=1):
    fn = "{}_{}.SCEN".format(*label)

    df.columns = df.columns.droplevel("MODEL").droplevel("SCENARIO")
    df.index = df.index.astype(int)
    
    fn
    df
    
    writer = MAGICCData()
    writer.df = df
    writer.metadata = {
        "header": "required for some reason\n\n"
    }
    writer.write(fn)

'MODEL-NAME-HYPHENS_SCENARIO-A-B-CDE-2.SCEN'

VARIABLE,BC_EMIS,C2F6_EMIS,CF4_EMIS,CH4_EMIS,CO2B_EMIS,CO2I_EMIS,CO_EMIS,N2O_EMIS,NH3_EMIS,NMVOC_EMIS,NOX_EMIS,OC_EMIS,SF6_EMIS,SOX_EMIS
TODO,SET,SET,SET,SET,SET,SET,SET,SET,SET,SET,SET,SET,SET,SET
UNITS,Mt BC/yr,kt C2F6/yr,kt CF4/yr,Mt CH4/yr,Mt CO2/yr,Mt CO2/yr,Mt CO/yr,kt N2O/yr,Mt NH3/yr,Mt VOC/yr,Mt NOx/yr,Mt OC/yr,kt SF6/yr,Mt SO2/yr
REGION,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD,WORLD
YEAR,,,,,,,,,,,,,,
2015,152.232300,-0.041672,9.171883,174.086943,5.784132,98.881333,164.542176,8.227947,201.585237,223.585251,188.452365,168.480964,4.430015,166.242029
2020,127.122154,5.356546,3.920455,190.914613,-1.819205,116.665714,207.880179,5.923396,222.346973,185.123285,207.703245,175.882816,3.886447,161.796137
2030,177.813834,1.744312,8.971443,173.510923,9.340767,94.997135,210.127141,-1.075648,189.346711,171.168715,206.074835,216.652539,5.770118,179.307805
2040,218.627907,4.660074,8.913939,183.605843,8.085740,157.824079,207.023793,-1.464650,229.721234,176.405342,242.040656,195.657868,2.465104,161.608984
2050,219.680082,4.051723,8.804623,172.782951,9.358825,130.577524,198.292512,9.521761,228.048844,195.762669,206.381987,158.507932,6.301458,180.567689
2060,190.530529,3.082902,7.593320,191.102728,6.954680,113.307226,179.222284,7.530809,184.317859,216.128281,187.982573,156.424602,0.385465,194.958677


ValueError: Could not determine scen special code for emissions ['BC', 'C2F6', 'CF4', 'CH4', 'CO2B', 'CO2I', 'CO', 'N2O', 'NH3', 'NMVOC', 'NOX', 'OC', 'SF6', 'SOX']

## Converting units

### An aside on how units work with Pint

We load units with Pint like so.

In [ ]:
ureg = pint.UnitRegistry()  # start a unit repository using the default variables
ureg.load_definitions('emissions_units.txt')  # load emissions units too
ureg._contexts  # show us which contexts we have available

In [ ]:
# define some variables
a = 1*ureg.C
b = 1*ureg.CO2
c = 3*ureg.N2O

In [ ]:
# they carry units with them
a
b
c

In [ ]:
# we can convert them to base units or to each other
b.to_base_units()
b.to('C')
c.to('N')

In [ ]:
%%expect_exception DimensionalityError
# if we try to do an invalid conversion, an error will be thrown
b.to('N2O')

In [ ]:
# however with a context, we can use metric conversions to 
# do our conversions
# AR4GWP12 is a made up metric where 1C = 20N
# hence 1 CO2 = 12/44 C = 12/44*20 N = 12/44*20*14/44 N2O

with ureg.context('AR4GWP12'):
    b
    b.to('N2O')
    12/44*20*14/44

### Converting pyam/openscm dataframes

This is a bit more fiddly so we wrap it in a function.

In [ ]:
ceds_pyam_df.head()

In [ ]:
def convert_variable_units(pyam_df, variable, target_units):
    output_df = pyam.IamDataFrame(pyam_df.data.copy())
    output_df.data.unit = output_df.data.unit.str.replace("-", "").replace("Mt CO2equiv/yr", "Mt CO2/yr")
    
    var_df = output_df.filter(variable=variable).data.copy()
    rest_df = output_df.filter(variable=variable, keep=False).data.copy()
    
    var_df = var_df.set_index(LONG_IDX).unstack(["variable", "unit"])
    var_df = var_df.pint.quantify(ureg, level=-1)
    
    for col in var_df:
        var_df[col] = var_df[col].pint.to("Gt C/yr")
    # annoying that pint dequantify destroys index
    old_index = var_df.index
    old_columns = var_df.columns
    var_df = var_df.pint.dequantify()
    var_df.index = old_index
    var_df.columns.names = old_columns.names + ['unit']

    var_df = var_df.stack().stack().reset_index()

    return pyam.IamDataFrame(pd.concat([var_df, rest_df]))

In [ ]:
ceds_pyam_df.filter(variable="Emissions*").head()

In [ ]:
ceds_pyam_df.filter(
    variable="Emissions|CO2*"
).head()
convert_variable_units(ceds_pyam_df, "Emissions|CO2","GtC/yr").filter(
    variable="Emissions|CO2*"
).head()